<a href="https://colab.research.google.com/github/Fahad-Blog/Data-Science-Portfolio/blob/main/RFM_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import datetime as dt

# 1. SETUP & MOCK DATA GENERATION
# In a real scenario, you would load this from Snowflake/BigQuery
data = {
    'CustomerID': np.random.randint(1000, 1100, 1000),
    'BookingID': range(1, 1001),
    'BookingDate': [dt.datetime(2025, 1, 1) + dt.timedelta(days=np.random.randint(0, 365)) for _ in range(1000)],
    'TotalAmount': np.random.uniform(100, 5000, 1000)
}
df = pd.DataFrame(data)

# 2. CALCULATING RFM METRICS
# We use a 'Snapshot Date' (typically 1 day after the last booking in the dataset)
snapshot_date = df['BookingDate'].max() + dt.timedelta(days=1)

rfm = df.groupby('CustomerID').agg({
    'BookingDate': lambda x: (snapshot_date - x.max()).days, # Recency
    'BookingID': 'count',                                   # Frequency
    'TotalAmount': 'sum'                                    # Monetary
})

# Rename for clarity
rfm.rename(columns={'BookingDate': 'Recency', 'BookingID': 'Frequency', 'TotalAmount': 'Monetary'}, inplace=True)

# 3. SCORING (1 to 5)
# Note: For Recency, a LOWER value is BETTER (more recent), so labels are reversed.
r_labels = range(5, 0, -1)
f_labels = range(1, 6)
m_labels = range(1, 6)

# We use rank method 'first' to handle non-unique bin edges in low-frequency data
rfm['R'] = pd.qcut(rfm['Recency'].rank(method='first'), q=5, labels=r_labels).astype(int)
rfm['F'] = pd.qcut(rfm['Frequency'].rank(method='first'), q=5, labels=f_labels).astype(int)
rfm['M'] = pd.qcut(rfm['Monetary'].rank(method='first'), q=5, labels=m_labels).astype(int)

# 4. SEGMENTATION LOGIC
def segment_me(df):
    r, f, m = df['R'], df['F'], df['M']
    if r >= 4 and f >= 4 and m >= 4:
        return 'Champions'
    elif f >= 4:
        return 'Loyalists'
    elif r >= 4 and f <= 2:
        return 'New Promising'
    elif r <= 2 and m >= 4:
        return 'At Risk (Big Spenders)'
    elif r <= 1:
        return 'Lost Customers'
    else:
        return 'Standard/Occasional'

rfm['Segment'] = rfm.apply(segment_me, axis=1)

# 5. OUTPUT
print(rfm['Segment'].value_counts())

Segment
Standard/Occasional       30
Loyalists                 25
Champions                 15
Lost Customers            14
New Promising             12
At Risk (Big Spenders)     4
Name: count, dtype: int64


In [ ]:
sql_query """

WITH base_metrics AS (
    -- STEP 1: Aggregate data at the Customer Level
    SELECT
        user_id,
        -- Recency: Days since the last confirmed booking
        DATE_DIFF(CURRENT_DATE(), MAX(booking_date), DAY) AS recency_days,
        -- Frequency: Count of unique bookings
        COUNT(DISTINCT booking_id) AS frequency_count,
        -- Monetary: Total Gross Booking Value (GBV)
        SUM(total_price_sar) AS total_monetary
    FROM `almosafer.raw.bookings`
    WHERE status = 'confirmed'
    GROUP BY 1
),

rfm_scores AS (
    -- STEP 2: Use NTILE to create quintiles (1-5) for each metric
    -- Note: For Recency, 5 is the 'best' (smallest number of days)
    SELECT
        *,
        NTILE(5) OVER (ORDER BY recency_days DESC) AS r_score,
        NTILE(5) OVER (ORDER BY frequency_count ASC) AS f_score,
        NTILE(5) OVER (ORDER BY total_monetary ASC) AS m_score
    FROM base_metrics
),

final_segmentation AS (
    -- STEP 3: Assign Personas based on Score Combinations
    SELECT
        *,
        CONCAT(r_score, f_score, m_score) AS rfm_combined,
        CASE
            WHEN r_score >= 4 AND f_score >= 4 AND m_score >= 4 THEN 'Champions'
            WHEN r_score >= 4 AND f_score >= 3 THEN 'Loyalists'
            WHEN r_score >= 4 AND f_score <= 2 THEN 'New Promising'
            WHEN r_score <= 2 AND m_score >= 4 THEN 'At Risk (Big Spenders)'
            WHEN r_score <= 1 THEN 'Lost Customers'
            ELSE 'Standard/Occasional'
        END AS persona
    FROM rfm_scores
)

-- Output the final dataset
SELECT * FROM final_segmentation

"""